# **Exercise 1**
étudiants : 
  João Pedro ARAUJO FERREIRA CAMPOS et Fabrício VELLONE




import libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

from torch.utils.tensorboard import SummaryWriter


# MNIST DATASET

Définition des paramètres

In [ ]:
# parameters and hyper parameters
# batch_size_train is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
# N, D_in, H, D_out = 64, 1000, 100, 10
D_in = 28 * 28 # Dimensoes das imagens (tem que ser)
H = 100 # H -> camada escondida (dimensao: 100 i.e: 10 neuronios / unidades)
D_out = 10 # 10 classes

num_epochs = 10

learning_rate = 0.01
w_decay = 1e-3
momentum = 0.9

batch_size_train = 64
batch_size_test = 1000

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

On télécharge les données de MNIST:

In [ ]:
#moyenne = 0.1307 ; std deviation = 0.3081 ; pour normalisation des données
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

Visualisation des données:

In [ ]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

import matplotlib.pyplot as plt

fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Ground Truth: {}".format(example_targets[i]))
  plt.xticks([])
  plt.yticks([])
fig

# Simple DNN

Define the simple network :

In [ ]:
class Net(nn.Module):
  # Toda rede é uma classe que extende nn.Module
  # No min tem que implementar esses 02 metodos
  # Pode ter varios forwards
  # auxilares: e.g: definir estatisticas, etc.



    # Censarios comuns
      # regressao -> nao usa funcao de ativacao na saida (equivale a func identidade)
      # classificacao binaria -> Regressao logistica
      # classificacao multiclasse -> softmax (transforma tudo em distribuicao) / depois tira o max

  def __init__(self):
    # construtor pode ser qqr coisa
    # boa pratica: Pasasr numero de classes como parametro
    super(Net, self).__init__()
    self.fc1 = nn.Linear(D_in, H) # FC: Fully connected (pq é linear)
    self.fc2 = nn.Linear(H, D_out) # Cada camada eh 01 matriz / isso é 01 jeito bonito de multiplicar matrizes
    # Cada camada é 01 matriz multiplicando
    # multiplicação de matriz é a coisa mais linear que tem
    # coisa linear é 'inutil'
    # RNA sem func ativacao é multiplicacao de matriz
    # func ativacao vai entrar na funcao forward

  def forward(self, x):
    # Voce que tem que implementar
    # Normalmente vc passa por todas as camadas
    
    # flatten
    x = x.view(-1,784)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

Initialisation


In [ ]:
model = Net()
model = model.to(DEVICE) #pour utiliser GPU (cuda)
# Aqui: Modelo ta na gpu
# Falta jogar as imagens pra gpu tb


Lancer l'optimizer

In [ ]:

# se quiser usar gpu tem que jogar tudo na gpu (nao esquecer)

criterion = nn.CrossEntropyLoss() # criterio -> funcao de custo (esses nomes sao padrao)
#  entropia cruzada pq tem n classes
# se fosse binario -> entropia binaria
# se fosse regressao -> minimos quadrados


# Funcao de custo é 01 superficie
# queremos achar o vale dela
# sgd: sthocastic gradient descent
# adam: outra opcao muito usada hoje em dia (tem mais uns 10 se quiser)
# decaimento: fator de regularizacao (manter a norma dos pesos prox de zero. Isso deixa o modelo mais simples e evita OF) / nao tem a ver com grad vanishing
# com 02 camadas n precisa preocupar com o vaninshing (eh mais comum em rnn. mlp & cnn sofre menos com isso)
# Momento: tem a ver com a inercia do gradiente (se eu detectar 01 plato é inutil andar pouquinho. Mas NAO é variar o passo)
# Estrategia de variar o passo é feito usando mais de 01 otimizador
# usa media dos gradientes passados pra nao deixar o gradiente variar demais
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay = w_decay, momentum=momentum)


Train the simple net

In [ ]:
train_losses = []
train_counter = []
writer = SummaryWriter('Res/Simple_net_train')
loss_l = []
for epoch in range(num_epochs):
  average_acc = 0
  average_loss = 0
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(DEVICE), target.to(DEVICE)
    optimizer.zero_grad() #zero the gradient so it doesn't get acumulated
    output = model(data) #forward pass
    # saida da softmax seria os logits (estatistica antes da predicao)

    #train accuracy
    y_hat = output.data.max(1, keepdim=True)[1] # certo era usar o softmax
    # logit
    # tensor == matriz dentro do pytorch
    # torch.tensor (todo dado tem que entrar assim. Tem que transformar imagens em torch tensor)
    train_acc = y_hat.eq(target.data.view_as(y_hat)).sum()
    average_acc += train_acc
    train_acc = train_acc /len(target.data)

    loss = criterion(output, target) #calculates the loss function (criterion == funcao de custo)
    average_loss += loss.item()

    loss.backward() #backward pass (backpropagation) (loss nao é 01 numero é 01 'nó' do grafo)
    optimizer.step() #updates the weights
    if batch_idx % 100 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      writer.add_scalar('Loss/train', loss.item(),
                        (epoch - 1) * len(train_loader) + batch_idx)
      writer.add_scalar('Accuracy/train', train_acc,
                        (epoch - 1) * len(train_loader) + batch_idx)
      
  writer.close()
  loss_l.append(loss.item())
  print(f"Epoch {epoch} average train loss: {average_loss / len(train_loader.dataset)}\n" f"Epoch {epoch} average train accuracy: {average_acc / len(train_loader.dataset)}\n")
        
# ----------------
plt.plot(loss_l)

Test the net

In [ ]:
test_losses = []
test_loss = 0
correct = 0

# modelo tem 02 modos: train (defaut) / eval
model.eval() # muda modo do modelo para avaliacao
# eh boa pratica usar isso sempre
# tem modelos com camadas que so existem durante o treinamento
# regularizacao
# eval congela tudo que so faz sentido o modelo fazer durante o treinamento
# dropout: quando elmina camadas durante o treinamento
# 

with torch.no_grad():
  # nao muda nada nomodlo. E mais pra economizar energia
  # inferencia: modelo ja esta treinado -> mudar modo do modelo & congenlar o gradiente
  for data, target in test_loader:
    data, target = data.to(DEVICE), target.to(DEVICE)
    output = model(data)
    test_loss += criterion(output, target)
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).sum()
test_loss /= len(test_loader.dataset)
test_losses.append(test_loss)
print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
  test_loss, correct, len(test_loader.dataset),
  100. * correct / len(test_loader.dataset)))

In [ ]:
interrupt #juste pour interrompre l'execution ici

# Convolutional NN

Définition du réseau CNN simple

In [ ]:
# CNN -> 02 etapas
# 01: extracao de caracteristicas (pelas convolucoes)
# 02: classificacao

# vc nk faz a conv sozinha -> conv, pooling + func de ativacao (geralmente nessa ordem)

# pooling: reduz a dimensao do tensor
# max pooling -> 

class CNN(nn.Module):
    #build the class, 1 channel and 10 classes (B&W images, 10 digits in the MNIST set) 
    def __init__(self, in_channels=1, num_classes=10):
        super(Net, self).__init__()
        
        # extracao de caracteristicas | kernel size -> tamanho do filtros
        self.conv1 = nn.Conv2d(in_channels, out_channels = 10, kernel_size=5) #2D convolutional layer |canais de entrada : 01 (p&b), senao seriam 03 (rgb) / dim de saida -> 10 (profundidade 10) / num de canais de saida === numero de filtros / nesse caso: 10 flitron de tamanho 5 
        
        # 01: 10 filtros em cima da mesma img (original)
        
        # 02: aplica 20 filtros em cima da 'imag' de 10 canais
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        

        # Na img original quero extrair 20 estatisticas
        # Na 2a quero ver em cima de 10, etc.

        # Reception field (vai aumentando a medida que a camada aprofunda)

        # conv sempre vem acompanhada do pooling (diminui a resolucao)


        # no inicio: 28 x 28 x 1
        # a medida que aprofunda vc vai tendo menos resolucao e mais profundidade

        # nao elimina camada inteira, apenas alguns neuronios (ajuda a regularizar)
        self.conv2_drop = nn.Dropout2d()

        # pra fazer a classificacao
        self.fc1 = nn.Linear(320, 50) #fully connected
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        
        # self.activation... (definir no init())


        x = F.relu(F.max_pool2d(self.conv1(x), 2)) # conv -> pool -> ...
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        
        x = x.view(-1, 320) # pq 320? Pq sim... | view ~ reshape
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

Implémentation de ResNet18

In [ ]:
# codigo fonte da resnet18

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
        
        super(Block, self).__init__()
        self.num_layers = 18

        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample

    def forward(self, x):
        identity = x

        x = self.conv1(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity # isso eh a skip connection | x final =  x passado pelas 
        x = self.relu(x)
        return x


class ResNet18(nn.Module):
    def __init__(self, block, image_channels, num_classes):

        super(ResNet18, self).__init__()
 
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # ResNetLayers
        self.layer1 = self.make_layers(block, 2, intermediate_channels=64, stride=1)
        self.layer2 = self.make_layers(block, 2, intermediate_channels=128, stride=2)
        self.layer3 = self.make_layers(block, 2, intermediate_channels=256, stride=2)
        self.layer4 = self.make_layers(block, 2, intermediate_channels=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x

    def make_layers(self, block, num_residual_blocks, intermediate_channels, stride):
        layers = []

        # res net -> residual net
        # todo: procurar diagrama dela
        # skip connection -> salto de camada (combate o vanishing)
        # qtd de blocos nao é livre mas é convencional / receomendado
        # backbone: extrator de caracteristicas (resnet50)
        # se extracao de caracteristicas nao for pre treinado (padrao da literatura) muito dificil de treinar
        # normalmente sao pre treinadas na image net
        # usar transfer learning
        # caracteristicas entre as imagens se repetem

        identity_downsample = nn.Sequential(nn.Conv2d(self.in_channels, intermediate_channels, kernel_size=1, stride=stride),
                                            nn.BatchNorm2d(intermediate_channels))
        layers.append(block(self.in_channels, intermediate_channels, identity_downsample, stride))
        self.in_channels = intermediate_channels  # 256
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels)) # 256 -> 64, 64*4 (256) again
        return nn.Sequential(*layers)

In [ ]:
def test():
    net = ResNet18(Block, 3, 1000)
    y = net(torch.randn(4, 3, 224, 224)).to(DEVICE)
    print(y.size())
    print(y)

test()

